In [2]:
import numpy as np
import pandas as pd
import string
import nltk, re, time
from string import punctuation
from nltk.corpus import stopwords
from tokenize import tokenize
from collections import defaultdict

from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf

In [3]:
train = pd.read_csv(r'C:\Users\Anjana Tiha\Drive D\Programming\Projects\Toxic Comment Classification Challenge\Toxic Comment Classification Challenge\all\train.csv')
test = pd.read_csv(r'C:\Users\Anjana Tiha\Drive D\Programming\Projects\Toxic Comment Classification Challenge\Toxic Comment Classification Challenge\all\test.csv')
test_labels = pd.read_csv(r'C:\Users\Anjana Tiha\Drive D\Programming\Projects\Toxic Comment Classification Challenge\Toxic Comment Classification Challenge\all\test_labels.csv')
sample_submission = pd.read_csv(r'C:\Users\Anjana Tiha\Drive D\Programming\Projects\Toxic Comment Classification Challenge\Toxic Comment Classification Challenge\all\sample_submission.csv')

In [4]:
columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# train['none'] = 1-train[columns].max(axis=1)
# train['comment_text'].fillna("unknown", inplace=True)
# test['comment_text'].fillna("unknown", inplace=True)

In [5]:
def clean_text(text, remove_stopwords=True):
    cached_stopwords = stopwords.words("english")
    text = text.lower()
    text = re.sub(r"[^a-z]", " ", text)
    text = re.sub(r" +", " ", text)
    text = ' '.join([word for word in text.split() if word not in cached_stopwords])
    text = ''.join([c for c in text if c not in punctuation])
    text = text.strip()
    text = text.split()
    return text 

In [6]:
tfidf = TfidfVectorizer(tokenizer=clean_text, min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1, smooth_idf=1 )
x = tfidf.fit_transform(train['comment_text'])
test_x = tfidf.transform(test['comment_text'])



In [7]:
# # print(data.isnull().sum(axis=1))
# X = features.values
# Y = target
# X = x+test_x
X = x
# Y = train['comment_text'] + test['comment_text']
# Y = train['comment_text']

# # print(features.info())

In [8]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics

# from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score
    
def model_evaluation(X, Y, X_test, y_test, splitter, model, report, details):
    accuracy = 0
    f1 = 0
    precision = 0
    recall = 0
    i=0
    if report:
        print("*"*50, " START ", "*"*50)
#         print("Spliter Description:")
#         print(splitter)
#         print("-"*100, "\n")
        print("Model Description:")
        print(model)
        print("-"*100,"\n")
      
    if splitter:
        for train_index, test_index in splitter.split(X, Y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]

            # model fitting
            model.fit(X_train, y_train)

            # prediction
            predict = model.predict(X_test)

            accuracy_temp = metrics.accuracy_score(y_test, predict)
            precision_temp = metrics.precision_score(y_test, predict, average="micro")
            recall_temp = metrics.recall_score(y_test, predict, average="micro")
            f1_temp = metrics.f1_score(y_test, predict, average="micro")
            hamming_loss = metrics.hamming_loss(y_test, predict)

            accuracy = accuracy + accuracy_temp
            precision = precision + precision_temp
            recall = recall+ recall_temp
            f1= f1 + f1_temp


    #         # evaluation scores
    #         explained_variance_score_temp = explained_variance_score(y_test, predict)
    #         mean_absolute_error_temp = mean_absolute_error(y_test, predict)
    #         mean_squared_error_temp = mean_squared_error(y_test, predict)
    #         mean_squared_log_error_temp = mean_squared_log_error(y_test, predict)
    #         median_absolute_error_temp = median_absolute_error(y_test, predict)
    #         r2_score_temp = r2_score(y_test, predict)

    #         explained_variance_score_val = explained_variance_score_val + explained_variance_score_temp
    #         mean_absolute_error_val = mean_absolute_error_val + mean_absolute_error_temp
    #         mean_squared_error_val = mean_squared_error_val + mean_squared_error_temp
    #         mean_squared_log_error_val = mean_squared_log_error_val + mean_squared_log_error_temp
    #         median_absolute_error_val = median_absolute_error_val + median_absolute_error_temp
    #         r2_score_val = r2_score_val + r2_score_temp

            if details:
                print("*"*25,  " ITERATION - ", i+1, "*"*25)
                #print("TRAIN:", train_index, "TEST:", test_index)
                print("accuracy_score", metrics.accuracy_score(y_test, predict))
                print("precision_score", metrics.precision_score(y_test, predict, average="micro"))
                print("recall_score", metrics.recall_score(y_test, predict, average="micro"))
                print("f1_score", metrics.f1_score(y_test, predict, average="micro"))
                print("hamming_loss", metrics.hamming_loss(y_test, predict))
            #     precision, recall, thresholds = metrics.precision_recall_curve(y_test, predict)
            #     print("average_precision_score", metrics.average_precision_score(y_test, predict, average="micro"))
            #     print("fbeta_score", metrics.fbeta_score(y_test, predict))
            #     print("roc_auc_score", metrics.roc_auc_score(y_test, predict, average="micro"))

    #             print("-"*35)
    #             print(metrics.classification_report(y_test, predict))
    #             print("-"*35)
    #             print("confusion Matrix:\n\n", metrics.confusion_matrix(y_test, predict))
    #             print("-"*35)
                print("\n")

            i+=1

        split_num = splitter.get_n_splits()

        split_num = splitter.get_n_splits()
        accuracy = accuracy/split_num
        precision = precision/split_num
        recall = recall/split_num
        f1 = f1/split_num

    else:
        model.fit(X, Y)

        # prediction
        predict = model.predict(X_test)

        accuracy = metrics.accuracy_score(y_test, predict)
        precision = metrics.precision_score(y_test, predict, average="micro")
        recall = metrics.recall_score(y_test, predict, average="micro")
        f1 = metrics.f1_score(y_test, predict, average="micro")
        hamming_loss = metrics.hamming_loss(y_test, predict)
    if report:
        if splitter:
            print("*"*50, " Average For", i+1, " Folds", "*"*50)
        print("\n")
        print("Average Accuracy Score: ", accuracy)
        print("Average pPrecision Score: ", precision)
        print("Average Recall Score: ", recall)
        print("Average F1 Score:", f1)

#         print('%50s%s' % ("Average explained_variance_score: ", explained_variance_score_val))
#         print('%50s%s' % ("Average mean_absolute_error: ", mean_absolute_error_val))
#         print('%50s%s' % ("Average mean_squared_error: ", mean_squared_error_val))
#         print('%50s%s' % ("Average mean_squared_log_error: ", mean_squared_log_error_val))
#         print('%50s%s' % ("Average median_absolute_error: ", median_absolute_error_val))
#         print('%50s%s' % ("Average r2_score: ", r2_score_val))
#         print("\n")
#         print("*"*100)
# #         print("*"*50, " END ", "*"*50)
    
#     return explained_variance_score_val, mean_absolute_error_val, mean_squared_error_val, mean_squared_log_error_val, median_absolute_error_val, r2_score_val
    return accuracy, precision, recall, f1

In [ ]:
from sklearn.model_selection import train_test_split


# classifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomTreesEmbedding, RandomForestClassifier, VotingClassifier)
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB 
from sklearn.neighbors import KDTree, KNeighborsClassifier, NearestNeighbors
from sklearn.neural_network import BernoulliRBM, MLPClassifier
from sklearn.svm import LinearSVC, NuSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# regressor
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomTreesEmbedding, RandomForestRegressor, VotingClassifier)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.isotonic import IsotonicRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ARDRegression, LinearRegression, LogisticRegression, LogisticRegressionCV, logistic_regression_path, HuberRegressor, PassiveAggressiveRegressor, RandomizedLogisticRegression, RANSACRegressor, SGDRegressor, TheilSenRegressor
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB 
from sklearn.neighbors import KDTree, KNeighborsRegressor, NearestNeighbors, RadiusNeighborsRegressor
from sklearn.neural_network import BernoulliRBM, MLPRegressor
from sklearn.svm import LinearSVR, NuSVR, SVR
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor




sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)


kf = KFold(n_splits = 5, random_state=None, shuffle =True)

classifiers = {
    "AdaBoostClassifier": AdaBoostClassifier(),
    "BernoulliNB": BernoulliNB(),
#     "BernoulliRBM": BernoulliRBM(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "ExtraTreesClassifier": ExtraTreesClassifier(),
#     "GaussianMixture": GaussianMixture(),
    "GaussianNB": GaussianNB(),
    "GaussianProcessClassifier": GaussianProcessClassifier(),
    "GradientBoostingClassifier": GradientBoostingClassifier(),
#     "KDTree": KDTree(),
    "KNeighborsClassifier": KNeighborsClassifier(3),
    "LogisticRegression": LogisticRegression(),
    "LinearSVC": LinearSVC(),
    "MLPClassifier": MLPClassifier(),
    "MultinomialNB": MultinomialNB(),
#     "NearestNeighbors": NearestNeighbors(),
    "NuSVC": NuSVC(),
    "QuadraticDiscriminantAnalysis": QuadraticDiscriminantAnalysis(),
    "RandomForestClassifier": RandomForestClassifier(),
    "SVC Linear": SVC(kernel="linear", C=0.025),
    "SVC": SVC(),
    "SVC Gamma": SVC(gamma=2, C=1)
#     VotingClassifier: VotingClassifier(),
}

regressors = {
    "AdaBoostRegressor": AdaBoostRegressor(),
#     "ARDRegression": ARDRegression(),
    "BaggingRegressor": BaggingRegressor(),
#     "BernoulliRBM": BernoulliRBM(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "ExtraTreesRegressor": ExtraTreesRegressor(),
    "ExtraTreeRegressor": ExtraTreeRegressor(),
#     "GaussianMixture": GaussianMixture(),
#     "GaussianNB": GaussianNB(),
    "GaussianProcessRegressor": GaussianProcessRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "HuberRegressor": HuberRegressor(),
#     "IsotonicRegression": IsotonicRegression(),
    "KernelRidge": KernelRidge(),
#     "KDTree": KDTree(),
#     "KNeighborsRegressor": KNeighborsRegressor(),
#     "LinearRegression": LinearRegression(), 
    "LogisticRegression": LogisticRegression(),
    "LogisticRegressionCV": LogisticRegressionCV(),
#     "logistic_regression_path": logistic_regression_path(),
    "LinearSVR": LinearSVR(),
    "MLPRegressor": MLPRegressor(),
#     "MultinomialNB": MultinomialNB(),
    "NuSVR": NuSVR(),
    "PassiveAggressiveRegressor": PassiveAggressiveRegressor(),
#     "QuadraticDiscriminantAnalysis": QuadraticDiscriminantAnalysis(),
    "RadiusNeighborsRegressor": RadiusNeighborsRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "RandomizedLogisticRegression": RandomizedLogisticRegression(),
    "RANSACRegressor": RANSACRegressor(),
    "SGDRegressor": SGDRegressor(),
    "SVR": SVR(),
    "TheilSenRegressor": TheilSenRegressor(),
}


    
# splitter = kf 
# splitter = sss
splitter = None
report = 1
details = 1
evaluation = {}

print("Spliter Description:")
print(splitter)
     

for comment_type in columns:
    print("comment Type: ", comment_type, "\n\n")
    for name in classifiers:
        evaluation_temp = []
        accuracy, precision, recall, f1 = model_evaluation(X, train[comment_type], test_x, test_labels[comment_type], splitter, classifiers[name], report, details)
        evaluation_temp.append(accuracy)
        evaluation_temp.append(precision)
        evaluation_temp.append(recall)
        evaluation_temp.append(f1)
        evaluation[name] = evaluation_temp
    rows_list = []
    for name in evaluation:
        rows_list.append([name]+evaluation[name])
                           
    evaluation_pd = pd.DataFrame(rows_list, columns=['model', 'accuracy', 'precision', 'recall', 'f1']) 
    print(evaluation_pd)


            
# for name in regressors:
#     evaluation_temp = []
    
#     explained_variance_score_val, mean_absolute_error_val, mean_squared_error_val, mean_squared_log_error_val, median_absolute_error_val, r2_score_val = model_evaluation(X, Y, splitter, regressors[name], report, details=None)
#     evaluation_temp.append(explained_variance_score_val)
#     evaluation_temp.append(mean_absolute_error_val)
#     evaluation_temp.append(mean_squared_error_val)
#     evaluation_temp.append(mean_squared_log_error_val)
#     evaluation_temp.append(median_absolute_error_val)
#     evaluation_temp.append(r2_score_val)
#     evaluation[name] = evaluation_temp
    

# rows_list = []
# for name in evaluation:
#     rows_list.append([name]+evaluation[name])
                           
# evaluation_pd = pd.DataFrame(rows_list, columns=['explained_variance_score',  'mean_absolute_error', 'mean_squared_error', 'mean_squared_log_error', 'median_absolute_error', 'r2_score']) 
# evaluation_pd = pd.DataFrame(rows_list, columns=['model', 'accuracy', 'precision', 'recall', 'f1']) 
# evaluation_pd


C:\Users\Anjana Tiha\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class RandomizedLogisticRegression is deprecated; The class RandomizedLogisticRegression is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


Spliter Description:
None
comment Type:  toxic 


**************************************************  START  **************************************************
Model Description:
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)
---------------------------------------------------------------------------------------------------- 



Average Accuracy Score:  0.38672272857851714
Average pPrecision Score:  0.38672272857851714
Average Recall Score:  0.38672272857851714
Average F1 Score: 0.38672272857851714
**************************************************  START  **************************************************
Model Description:
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
---------------------------------------------------------------------------------------------------- 



Average Accuracy Score:  0.28835104854926746
Average pPrecision Score:  0.28835104854926746
Average Recall Score

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import figure


figure(num=None, figsize=(14, 6), dpi=250)

labels= ['accuracy', 'precision', 'recall', 'f1']
ax = plt.subplot(111)

for n in range(0,4):
    plt.plot([name for name in evaluation],[evaluation[name][n] for name in evaluation], label = labels[n])

leg = plt.legend(loc='best', ncol=2, mode="expand", shadow=True, fancybox=True)
plt.xticks(rotation=45)
# leg.get_frame().set_alpha(0.5)
plt.legend()
ax.tick_params(labelsize='large', width=5)
ax.grid(True, linestyle='-.')

plt.tight_layout()
plt.xlabel('x label')
plt.ylabel('y label')

plt.title("TITLE")
plt.show()
